To-Do: entire workflow
chemdraw parse -> combinatorial expansion -> conformer generation -> aso description -> post-processing/dimension-reduction/analysis -> plot/output data

1. ChemDraw Parsing

In [1]:
import molli as ml
import molli.visual
import subprocess

# This is a failsafe in case openbabel aint installed
import openbabel

In [2]:
cores = ml.files.box_cores_test_1
subs = ml.files.box_substituents_test_1

In [3]:
subprocess.run(['molli', 'parse', '--hadd', f'{cores}', '-o', '../ncsa-testing/ncsa-testing-data/BOX_cores_new_env.mlib', "--overwrite"])
subprocess.run(['molli', 'parse', '--hadd', f'{subs}', '-o', '../ncsa-testing/ncsa-testing-data/BOX_subs_new_env.mlib', "--overwrite"])

100%|██████████| 15/15 [00:00<00:00, 277.99it/s]


CompletedProcess(args=['molli', 'parse', '--hadd', '/home/shvedalx/dev/molli/molli/files/BOX_substituents_test_1.cdxml', '-o', '../ncsa-testing/ncsa-testing-data/BOX_subs_new_env.mlib', '--overwrite'], returncode=0)

In [4]:
m_core = ml.MoleculeLibrary('ncsa-testing-data/BOX_cores_new_env.mlib')
print(len(m_core))
m_core[0]

5


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='1', formula='C10 H12 Br2 N2 O2 Unknown2')

In [5]:
m_subs = ml.MoleculeLibrary('ncsa-testing-data/BOX_subs_new_env.mlib')
print(len(m_subs))
m_subs[0]

15


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='3', formula='C3 H7 Unknown1')

2. Combinatorial Expansion

In [6]:
subprocess.run(
    [
        'molli',
        'combine',
        'ncsa-testing-data/BOX_cores_new_env.mlib',
        '-s',
        'ncsa-testing-data/BOX_subs_new_env.mlib',
        '-j',
        '96', 
        '-o', 
        'ncsa-testing-data/test_combine_new_env.mlib', 
        '-a', 
        'A1', 
        '--obopt', 
        'uff',
        '-m',
        'same',
        "--overwrite"
    ]
)


Will create a library of size 75


100%|██████████| 75/75 [00:02<00:00, 34.85it/s]


CompletedProcess(args=['molli', 'combine', 'ncsa-testing-data/BOX_cores_new_env.mlib', '-s', 'ncsa-testing-data/BOX_subs_new_env.mlib', '-j', '96', '-o', 'ncsa-testing-data/test_combine_new_env.mlib', '-a', 'A1', '--obopt', 'uff', '-m', 'same', '--overwrite'], returncode=0)

In [7]:
combined = ml.MoleculeLibrary('ncsa-testing-data/test_combine_new_env.mlib')
print(len(combined))
combined["1_3_3"]

75


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [8]:
combined["3_6_6"]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Molecule(name='3_6_6', formula='C30 H38 Br2 N2 O2')

3. Conformer Generation

In [9]:
subprocess.run(['molli', 
                'conformers', 
                'ncsa-testing-data/test_combine_new_env.mlib', 
                '-n', 
                '50', 
                '-o', 
                'ncsa-testing-data/test_conformers_new_env.mlib', 
                '-t', 
                '-j', ### !!!!!! Number of jobs. Please scale down if host system has fewer cores. defaults to os.cpu_count()//2  !!!!! ###
                '96',
                "--overwrite"
                ])

100%|██████████| 75/75 [02:11<00:00,  1.76s/it]


CompletedProcess(args=['molli', 'conformers', 'ncsa-testing-data/test_combine_new_env.mlib', '-n', '50', '-o', 'ncsa-testing-data/test_conformers_new_env.mlib', '-t', '-j', '96', '--overwrite'], returncode=0)

In [10]:
clib = ml.ConformerLibrary('ncsa-testing-data/test_conformers_new_env.mlib')
print(len(clib))

75


In [11]:
i = 0
for conf in clib:
    i += conf.n_conformers
    # print(conf)
print(str(i) + ' conformers in library')

8028 conformers in library


In [13]:
print(clib[0])
clib[0][0]

ConformerEnsemble(name='1_3_3', formula='C16 H26 Br2 N2 O2', n_conformers=150)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [14]:
clib[0][1]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='1_3_3', formula='C16 H26 Br2 N2 O2')

In [15]:
clib[42][0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='3_6_6', formula='C30 H38 Br2 N2 O2')

In [16]:
clib[42][24]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Conformer(name='3_6_6', formula='C30 H38 Br2 N2 O2')

3.5. Create grid

In [17]:
subprocess.run(['molli', 
                'grid', 
                '--mlib', 
                'ncsa-testing-data/test_conformers_new_env.mlib', 
                '-o', 
                'ncsa-testing-data/grid_new_env.npy'
                ])

(7106, 3)


100%|██████████| 75/75 [00:00<00:00, 1637.67it/s]


CompletedProcess(args=['molli', 'grid', '--mlib', 'ncsa-testing-data/test_conformers_new_env.mlib', '-o', 'ncsa-testing-data/grid_new_env.npy'], returncode=0)

4. ASO description of molecules

In [18]:
subprocess.run(['molli', 
                'gbca', 
                'aso', 
                'ncsa-testing-data/test_conformers_new_env.mlib', 
                '-g', 
                'ncsa-testing-data/grid_new_env.npy', 
                '-o', 
                'ncsa-testing-data/aso_new_env.h5'
                ])
# tqdm looks messed up

Allocating storage for descriptors
Will compute descriptor ASO using 128 cores.
Grid shape: (7106, 3)


Loading batches of conformers: 0it [00:00, ?it/s]
Submitting calculations:  87%|████████▋ | 65/75 [00:11<00:02,  3.94it/s]
                                                                        

Gathering calculation results:   0%|          | 0/75 [00:00<?, ?it/s]

Loading batches of conformers: 0it [00:13, ?it/s]                    
/home/shvedalx/dev/molli/molli/lib_gen/test_aso/alt_aso.py:51: UserWarning: Invalid filepath: Unable to create dataset (name already exists)
  warnings.warn(f'Invalid filepath: {exp!s}')


CompletedProcess(args=['molli', 'gbca', 'aso', 'ncsa-testing-data/test_conformers_new_env.mlib', '-g', 'ncsa-testing-data/grid_new_env.npy', '-o', 'ncsa-testing-data/aso_new_env.h5'], returncode=0)

In [ ]:
# subprocess.run(['molli', 'grid', '--mlib', '../../out_conformers1/test_conformers.mlib', '-o', 'molli/lib_gen/test_aso'])


In [ ]:
#subprocess.run(['molli', 'descriptors', '../../out_conformers1/test4_conformers.mlib', '-o', '../../test_aso_data/test_aso.h5'])

5. post-processing/dimension-reduction/analysis

In [20]:
subprocess.run(         # check functionality for plotting and pca
     [                   # should be better way to implement post_processing stuff
         'molli', 
         'cluster', 
         'ncsa-testing-data/aso_new_env.h5', 
         '-o', 
         'ncsa-testing-data/new_env_data3', 
         '-v',
         '0',
         '-c',
         '0.8',
     ]
 )

Traceback (most recent call last):
  File "/home/shvedalx/dev/molli/molli/__main__.py", line 112, in main
    requested_module = import_module(f"molli.scripts.{cmd}")
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/shvedalx/mambaforge/envs/molli-dev/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/shvedalx/dev/molli/molli/scripts/cluster.py", line 10, in <module>
    import mol

CompletedProcess(args=['molli', 'cluster', 'ncsa-testing-data/aso_new_env.h5', '-o', 'ncsa-testing-data/new_env_data3', '-v', '0', '-c', '0.8'], returncode=1)